In [8]:
from config import dogs_vs_cats_config as config
from loader_util.preprocessing import ImageToArrayPreprocessor, \
    SimplePreProcessor, PatchPreprocessor,MeanPreprocessor
from loader_util.callbacks import TrainingMonitor
from loader_util.io import HDF5DatasetGenerator
from loader_util.nn.conv import AlexNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import json
import os

In [9]:
# construct the training the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode='nearest')

In [10]:
# load the RGB means for he train set
f = open(config.DATASET_MEAN)
file_contents = f.read()
f.close()
file_contents

'{"R": 125.17531584701538, "G": 116.08281515960694, "B": 106.28272320175171}'

In [11]:
means = json.loads(file_contents)
batch_size = 16
epoch_num = 50

# initalise the image preprocessors
sp = SimplePreProcessor(227, 227)
pp = PatchPreprocessor(227, 227)
mp = MeanPreprocessor(means['R'], means['G'], means['G'])
iap = ImageToArrayPreprocessor()

In [12]:
# initialise the train and valid dataset generators
trainGen = HDF5DatasetGenerator(config.TRAIN_HDF5,
                                batchSize=batch_size,
                                aug=aug,
                                preprocessors=[pp, mp, iap],
                                classes=2)
valGen = HDF5DatasetGenerator(config.VALID_HDF5,
                              batchSize=batch_size,
                              preprocessors=[sp, mp, iap],
                              classes=2)

In [13]:
# initalise optimiser
opt = Adam(lr=1e-3)
model = AlexNet.build(227, 227, depth=3, classes=2, reg=0.0002)
model.compile(loss='binary_crossentropy', optimizer=opt, 
                  metrics=['accuracy'])

# construct callbacks
path = os.path.sep.join([config.OUTPUT_PATH, f'{os.getpid()}.png'])
callbacks = [TrainingMonitor(path)]

In [14]:
# train the net
H = model.fit_generator(
    trainGen.generator(),
    steps_per_epoch=trainGen.numImages // batch_size,
    validation_data=valGen.generator(),
    validation_steps=valGen.numImages // batch_size,
    epochs=epoch_num,
    max_queue_size=10,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50
1250/1250 [==============================] - 967s 773ms/step - loss: 3.5223 - accuracy: 0.5273 - val_loss: 2.3185 - val_accuracy: 0.5625

KeyboardInterrupt: 

In [ ]:
model.save(config.MODEL_PATH, overwrite=True)

# close the datasets
trainGen.close()
valGen.close()

'{"R": 125.17531584701538, "G": 116.08281515960694, "B": 106.28272320175171}'